In [1]:
import pandas as pd
from sklearn import cross_validation as cv
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.preprocessing import LabelEncoder, scale
from sklearn.metrics import accuracy_score
from sklearn import ensemble
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [18]:
test_data = pd.read_csv('all/test.csv')
train_data = pd.read_csv('all/train.csv')
print(train_data.shape, test_data.shape)


(891, 12) (418, 11)


In [19]:
X = train_data.iloc[: , 2:]
y = train_data.Survived
X_test_1 = test_data.iloc[: , 1:]
X_test_1[:10]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [4]:
print (X.shape, y.shape, X_test_1.shape)

(891, 10) (891,) (418, 10)


In [5]:
#X = pd.get_dummies(X)

In [6]:
X_1 = X.drop(['Name', 'Ticket'], axis = 1)
X_2 = X_1.fillna(value = {'Age' : 10, 'Embarked' : 'Q'})


In [7]:
X_test_1 = X_test_1.drop(['Name', 'Ticket', ], axis = 1)
X_test_1 = X_test_1.fillna(value = {'Age' : 10, 'Embarked' : 'Q', 'Fare' : 10})

Get dummy

In [17]:
#X_test_1
X_2 = pd.get_dummies(X_2)
X_test_1 = pd.get_dummies(X_test_1)
X_test_1[:10]


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A11,Cabin_A18,Cabin_A21,...,Cabin_F E46,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,3,14.0,0,0,9.2250,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,3,30.0,0,0,7.6292,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,2,26.0,1,1,29.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,3,18.0,0,0,7.2292,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,3,21.0,2,0,24.1500,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
#X_2 = scale(X_2)
#X_test_1 = scale(X_test_1)
#X_2 = pd.DataFrame(X_2)


In [10]:
X_train, X_test, y_train, y_test = cv.train_test_split(X_2, y, test_size = 0.2)

In [11]:
LG = LogisticRegression(class_weight = 'balanced')
LG.fit(X_train, y_train)
RC = RidgeClassifier()
RC.fit(X_train, y_train)

rf = ensemble.RandomForestClassifier(random_state=1, n_estimators=300, min_samples_split=6, min_samples_leaf=2)
rf.fit(X_train, y_train)
#scores = cross_val_score(rf, X_train, y_train, cv = 4, n_jobs=-1)
#scores.mean()


#tree = DecisionTreeClassifier()
#Bag = BaggingClassifier(tree, n_estimators = 100)
#Bag.fit(X_train, y_train)
#

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [12]:

y_bagg_pred = Bag.predict(X_test_1)
y_pred = Bag.predict(X_test)

arr = np.arange(892,1310)
d = {'PassengerId' : arr, 'Survived' : y_bagg_pred}
otvet = pd.DataFrame(data = d)
otvet.to_csv('otvet_bagg.csv', index = False)

accuracy_score(y_test, y_pred)

NameError: name 'Bag' is not defined

In [13]:


#modelXGB = xgb.XGBRegressor().fit(X_train, y_train)

#y_xgb_pred = xgb.predict(X_test_1)
#y_pred = xgb.predict(X_test)

#arr = np.arange(892,1310)
#d = {'PassengerId' : arr, 'Survived' : y_xgb_pred}
#otvet = pd.DataFrame(data = d)
#otvet.to_csv('otvet_xgb.csv', index = False)

#accuracy_score(y_test, y_pred)

In [14]:
y_rf_pred = rf.predict(X_test_1)
y_pred = rf.predict(X_test)


arr = np.arange(892,1310)
d = {'PassengerId' : arr, 'Survived' : y_rf_pred}
otvet = pd.DataFrame(data = d)
otvet.to_csv('otvet_rf.csv', index = False)

accuracy_score(y_test, y_pred)

ValueError: Number of features of the model must match the input. Model n_features is 157 and input n_features is 86 

In [15]:
y_pred_rc = RC.predict(X_test_1)

y_pred = RC.predict(X_test)

arr = np.arange(892,1310)
d = {'PassengerId' : arr, 'Survived' : y_pred_rc}
otvet = pd.DataFrame(data = d)
otvet.to_csv('otvet_rc.csv', index = False)

accuracy_score(y_test, y_pred)

ValueError: X has 86 features per sample; expecting 157

In [16]:
y_pred_log = LG.predict(X_test_1)

y_pred = LG.predict(X_test)

arr = np.arange(892,1310)
d = {'PassengerId' : arr, 'Survived' : y_pred_log}
otvet = pd.DataFrame(data = d)
otvet.to_csv('otvet_log.csv', index = False)

accuracy_score(y_test, y_pred)

ValueError: X has 86 features per sample; expecting 157